In [5]:
import torch
import os

if torch.cuda.is_available():
    device = torch.device('cuda:2')

ModuleNotFoundError: No module named 'torch'

In [37]:
FILE_ROOT = '/home/i-hsuan/mb/data'

EPOCH = 15
BATCH_SIZE = 128

LEARING_RATE = 0.001
MOMENTUN = 0.999

In [32]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = datasets.ImageFolder(root=FILE_ROOT, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [41]:
import torchvision.models as models

model = models.mobilenet_v3_large(pretrained=True).to(device)
for param in model.parameters():
    param.requires_grad = False

model.classifier[3] = torch.nn.Sequential(
                        torch.nn.Linear(model.classifier[3].in_features, 1024).to(device),
                        torch.nn.Hardswish(),
                        torch.nn.Dropout(p=0.5),
                        torch.nn.Linear(1024, 512),
                        torch.nn.Hardswish(),
                        torch.nn.Dropout(p=0.5),
                        torch.nn.Linear(512, 4),
                        ).to(device)

model.train()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [42]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier[3].parameters(), lr=LEARING_RATE, momentum=MOMENTUN)

for param in model.classifier[3].parameters():
    param.requires_grad = True
for epochs in range(EPOCH):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):  # 假設 data_loader 是您的訓練數據
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 0:
            print(f'Epoch [{epochs+1}/{EPOCH}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 10:.4f}')

    print(f"Epoch {epochs+1}, Loss: {running_loss / len(train_loader)}")

# for param in model.features[8].parameters():
#     param.requires_grad = True
# for epochs in range(EPOCH - 2):
#     running_loss = 0.0
#     for i, (inputs, labels) in enumerate(train_loader, 0):  # 假設 data_loader 是您的訓練數據
#         inputs, labels = inputs.to(device), labels.to(device)

#         optimizer.zero_grad()

#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         if i % 100 == 0:
#             print(f'Epoch [{epochs+1}/{EPOCH}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / 10:.4f}')

#     print(f"Epoch {epochs+1}, Loss: {running_loss / len(train_loader)}")


print('Done')

Epoch [1/15], Step [1/50], Loss: 0.1385
Epoch 1, Loss: 1.3786974215507508
Epoch [2/15], Step [1/50], Loss: 0.1363
Epoch 2, Loss: 1.3324241614341736
Epoch [3/15], Step [1/50], Loss: 0.1267
Epoch 3, Loss: 1.2160947680473329
Epoch [4/15], Step [1/50], Loss: 0.1181
Epoch 4, Loss: 1.022530972957611
Epoch [5/15], Step [1/50], Loss: 0.0927
Epoch 5, Loss: 0.9008034706115723
Epoch [6/15], Step [1/50], Loss: 0.0946
Epoch 6, Loss: 0.8676581782102585
Epoch [7/15], Step [1/50], Loss: 0.0618
Epoch 7, Loss: 0.8656150579452515
Epoch [8/15], Step [1/50], Loss: 0.0842
Epoch 8, Loss: 0.7913598138093948
Epoch [9/15], Step [1/50], Loss: 0.0763
Epoch 9, Loss: 0.6967550247907639
Epoch [10/15], Step [1/50], Loss: 0.0556
Epoch 10, Loss: 0.7036971133947373
Epoch [11/15], Step [1/50], Loss: 0.0894
Epoch 11, Loss: 0.6805179297924042
Epoch [12/15], Step [1/50], Loss: 0.0543
Epoch 12, Loss: 0.6344170248508454
Epoch [13/15], Step [1/50], Loss: 0.0670
Epoch 13, Loss: 0.625884798169136
Epoch [14/15], Step [1/50], Loss

In [43]:
model.eval()

val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_loss /= len(val_loader)
accuracy = 100 * correct / total
print(f"Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")

with torch.no_grad():
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_loss /= len(val_loader)
accuracy = 100 * correct / total
print(f"Testing Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")

Validation Loss: 0.8069, Accuracy: 69.88%
Testing Loss: 1.5985, Accuracy: 81.78%


In [44]:
torch.save(model, '/home/i-hsuan/mb/model.pth')

In [ ]:
import cv2
import torch
from torchvision import transforms
from PIL import Image
import numpy as np

# 定義裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義與訓練一致的數據變換
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 加載情緒分類模型
model = torch.load("MobileNetV3_large_val_acc_0.69.pth", map_location=device)
model = model.to(device)
model.eval()

# 定義狗狗情緒類別
class_names = ["angry", "happy", "relax", "sad"]

# 打開影片文件
video_path = "dog_video.mp4"  # 替換為你的影片路徑
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Cannot open video file.")
    exit()

# 定義縮放比例（如 50% 大小）
scale_percent = 50

# 處理影片每一幀
while cap.isOpened():
    ret, frame = cap.read()  # 獲取影片幀
    if not ret:
        break  # 若影片結束，退出循環

    # 縮小畫面
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    resized_frame = cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)

    # 將縮小後的圖像轉為 PIL 圖像
    pil_image = Image.fromarray(cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB))

    # 預處理圖像
    input_tensor = transform(pil_image).unsqueeze(0).to(device)

    # 進行情緒推理
    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class = class_names[predicted.item()]

    # 在縮小畫面上顯示預測類別
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(resized_frame, f"Emotion: {predicted_class}", (10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # 顯示處理後的縮小畫面
    cv2.imshow("Dog Emotion Detection", resized_frame)

    # 按下 'q' 鍵退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()
